In [2]:
# Define SCHOOL, data_path, and output_path
SCHOOL = "McGill"
data_path = f"bias_processing/data/1/{SCHOOL.lower()}_dataset.csv"
output_path = f"bias_processing/data/2/{SCHOOL.lower()}_dataset_granularity.csv"

In [3]:
"""
Load in a csv from Sentiment_Dataset_Maker and add 4x3x3 columns
4 topics ("Israel", "Palestine", "India", "China")
3 hypotheses for sentiment (Positive, Negative, Neutral)
3 levels of granularity
Compute sentiment for entire article
Summarizes each paragraph using an ML summarizing model, and join those summaries to one body of text. Compute sentiment for this new article version.
Summarize the entire article in one go using the same ML model. Compute sentiment for this new article version
Save a new csv with these added columns
"""

'\nLoad in a csv from Sentiment_Dataset_Maker and add 4x3x3 columns\n4 topics ("Israel", "Palestine", "India", "China")\n3 hypotheses for sentiment (Positive, Negative, Neutral)\n3 levels of granularity\nCompute sentiment for entire article\nSummarizes each paragraph using an ML summarizing model, and join those summaries to one body of text. Compute sentiment for this new article version.\nSummarize the entire article in one go using the same ML model. Compute sentiment for this new article version\nSave a new csv with these added columns\n'

In [4]:
%pip install nltk pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import os
import nltk
os.environ['NLTK_DATA'] = r"\Users\stacy\Desktop\Olami Project\Olami"
nltk.download('vader_lexicon')
nltk.download('stopwords')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\adaml\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\adaml\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [36]:
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from statistics import mean

# Instantiate the sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Function to return the sentiment of a text
def get_sentiment(text, granularity, keyword, model="nltk_sia", method='avg'):
    if model == "nltk_sia":
        # Output is a dict containing {'neg','pos','neu','composition'}. First three are needed for all future functionality
        def get_model_scores(text):
            scores = sia.polarity_scores(text)
            return scores
    if granularity == 'article':
        scores = get_model_scores(text)
        return scores['neg'], scores['pos'], scores['neu']
    elif granularity in ['paragraph','sentence']:
        if granularity == 'paragraph':
            # Calculate the polarity scores for each paragraph and store them in a list
            # TODO: Revise and check paragraph splitting, may have issues with article splitting
            listed_scores = [get_model_scores(paragraph) for paragraph in text.split('\n') if paragraph]
        elif granularity == 'sentence':
            listed_scores = [get_model_scores(sentence) for sentence in nltk.sent_tokenize(text)]

        # Transpose the list of dictionaries to separate the values for each key
        transposed_scores = list(zip(*[d.values() for d in listed_scores]))

        # Find the maximum value for each key using the max function
        if method == 'max':
            ranked_scores = [max(scores) for scores in transposed_scores]
        elif method == 'avg':
            ranked_scores = [sum(scores) / len(scores) for scores in transposed_scores]

        # Create a dictionary with the corresponding keys and maximum values
        result_dict = dict(zip(sia.polarity_scores(text).keys(), ranked_scores))

        return result_dict['neg'], result_dict['pos'], result_dict['neu']

# Load the csv
df = pd.read_csv(data_path)

# Process sentiment analysis
for granularity in ['article','paragraph','sentence']:
    print(granularity)
    df[f'{granularity}_neg'], df[f'{granularity}_pos'], df[f'{granularity}_neu'] = zip(*df.apply(lambda row: get_sentiment(row['article'], granularity, row['keyword']), axis=1))
        

# Save the output DataFrame into a new CSV file
df.to_csv(output_path, index=False)

article
paragraph
sentence
